# Querying the index and getting the candidates docs
written by Yige Wen

----

In [0]:
from whoosh.qparser import QueryParser
from whoosh import scoring
import time

from whoosh.index import open_dir
ix = open_dir('title_index_analyzer')

In [0]:
from whoosh import analysis
analyzer1=analysis.KeywordAnalyzer(lowercase=True)
title = "Robert_J._O'Neill_-LRB-U.S._Navy_SEAL-RRB-"


def delete_bracket(title):
    result = []
    tokens = ' '.join(title.split("_"))
#     print([t.text for t in analyzer(tokens)])
    tokens = [t.text for t in analyzer1(tokens)]
    for t in tokens:
        if t[:5] != '-lrb-' and t[-5:] != '-rrb-':
            result.append(t)
    return ' '.join(result)

print(delete_bracket(title))

def pre_process(title):
    tokens = title.replace("_", ' ')
    tokens = tokens.replace("-", ' ')
    tokens = tokens.replace("COLON", '')
    tokens = tokens.split()
    stack = [tokens[0]]
    
    if stack[0] == 'LRB':
        tokens = ' '.join(tokens)
        tokens = tokens.replace("LRB", '')
        tokens = tokens.replace("RRB", '')
        return tokens.split()
    
    for i in range(1, len(tokens)):
        head = stack[-1]
        if head == 'LRB':
            if tokens[i] == 'RRB':
                stack.pop()
            elif tokens[i] == 'LRB':
                stack.append(tokens[i])
            continue
        stack.append(tokens[i])
    return stack

print(pre_process(title))

robert j. o'neill navy
['Robert', 'J.', "O'Neill"]


In [0]:
def get_rid_of_s(entity):
    ent = entity[:]
    ent = ent.split(' ')
    if ent[-1] == "'" or ent[-1] == "'s":
        ent.pop()
    return ' '.join(ent)

get_rid_of_s("Carey Hayes '")

'Carey Hayes'

In [0]:
import json

with open('NER-CF-test-unlabelled.json','r') as f:
    data = json.load(f)

# with open('NER-CF-test-unlabelled.json','r') as f:
#     data = json.load(f)
    
print(len(data))
    
for k,v in data.items():
    for ent in v['entities']:
        x = get_rid_of_s(ent)
        if not x in v['entities']:
            v['entities'].append(x)
        v['entities'] = list(set(v['entities']))
        
    temp = []
    for ent in v['entities']:
        ent = ent.split(' ')
        if len(ent)>1 or ent[0].isalpha():
            temp.append(' '.join(ent))
    v['entities'] = temp
print(data['63631'])
# print(data['204443'])


14997
{'claim': 'In the End was released in 1700 .', 'entities': ['In the End']}


In [0]:
from collections import Counter

def filter_title(title, claim):
    title_counts = Counter(title)
    claim_counts = Counter(claim)
    for k,v in title_counts.items():
        if v > claim_counts[k]:
            return False
    return True

In [0]:
from whoosh.qparser import QueryParser

from whoosh import analysis
analyzer2=analysis.StandardAnalyzer(minsize=1, stoplist=[])

import time
start = time.clock()

doc_results = {}
with ix.searcher() as searcher1:
    i = 1
    for k,v in data.items():
#         if i == 2:
#             break
#         i += 1
        doc_result = {}
        evidence = []
#         set_claim = set([t.text for t in analyzer2(v['claim'])])
        claim = [t.text for t in analyzer2(v['claim'])]
        entities = v['entities']
        titles = []
        for q in entities:
            query = QueryParser("content", ix.schema).parse(q)
            result = searcher1.search(query, limit=20)
            ##
            for hit in result:
                real_title = [t.text for t in analyzer2(' '.join(pre_process(hit['title'])))]
#                 print(real_title)
#                 print(set_claim)
#                 if set(real_title).issubset(set_claim):
#                     titles.append(hit['title'])
                if filter_title(real_title,claim):
                    titles.append(hit['title'])
            ##
#             titles += [hit['title'] for hit in result]
            
        titles = list(set(titles))
        for t in titles:
            evidence.append([t, 0])
        
#         doc_result['claim'] = ' '.join(v['claim'])
        doc_result['label'] = 'SUPPORTS'
        doc_result['evidence'] = evidence
        doc_results[k] = doc_result

with open("result_limit_20-NER-CF-test-unlabelled.json","w") as w:
    json.dump(doc_results, w, indent=2, separators=(', ', ': '))
        
# with open("result_limit_20-NER-CF-test-unlabelled.json","w") as w:
#     json.dump(doc_results, w, indent=2, separators=(', ', ': '))

In [0]:
analyzer2=analysis.StandardAnalyzer(minsize=1)
print([t.text for t in analyzer2( "See.SZA.Run")])

['see.sza.run']


In [0]:
with open('result_limit_20-NER-CF-test-unlabelled.json','r') as f:
    data = json.load(f)
print(len(data))

for k,v in data.items():
    v['label'] = 'SUPPORTS'

with open('testoutput.json','w') as w:
    json.dump(data, w, indent=2, separators=(', ', ': '))

14997


In [0]:
inputJson = 'devset.json'
with open(inputJson,'r') as f:
    data = json.load(f)
print(len(data))

out = {}
for k,v in data.items():
    if v['label'][0] != 'N':
        out[k] = v
print(len(out))

with open('devset_supp_refu.json','w') as w:
    json.dump(out, w, indent=2, separators=(', ', ': '))

In [0]:
from whoosh.qparser import QueryParser
from whoosh import scoring
import time

start = time.clock()

entities = [
      "See SZA Run"
    ]
# entities = list(set([" ".join(pre_process(e)) for e in entities]))

with ix.searcher() as searcher1:
    print(time.clock() - start)
    for q in entities:
        parser = QueryParser("content", ix.schema)
        query = parser.parse(q)
        print(query)
        results = searcher1.search(query)
        for hit in results:
            print(hit['title'], hit['wiki'])
        print(time.clock() - start)
        
elapsed = (time.clock() - start)
print("Time used:",elapsed)

1.639811
(content:see AND content:sza AND content:run)
1.6494109999999997
Time used: 1.7380630000000004


In [0]:
from whoosh.qparser import QueryParser

from whoosh import analysis
analyzer2=analysis.StandardAnalyzer(minsize=1)

doc_results = {}
with ix.searcher() as searcher1:

    k = '59356'
    v =  data[k]
    
    doc_result = {}
    evidence = []
    set_claim = set([t.text for t in analyzer2(v['claim'])])
    entities = v['entities']
    titles = []
    for q in entities:
        query = QueryParser("content", ix.schema).parse(q)
        result = searcher1.search(query)
        ##
        for hit in result:
            real_title = [t.text for t in analyzer2(' '.join(pre_process(hit['title'])))]
            print(real_title)
            print(set_claim)
            print(set(real_title).issubset(set_claim))
            if set(real_title).issubset(set_claim):
                titles.append(hit['title'])
        ##
#             titles += [hit['title'] for hit in result]

    titles = list(set(titles))
    for t in titles:
        evidence.append([t, 0])
        
#         doc_result['claim'] = ' '.join(v['claim'])
        doc_result['label'] = 'SUPPORTS'
        doc_result['evidence'] = evidence
        doc_results[k] = doc_result
        
print(doc_results[k])

In [0]:
from allennlp.predictors.predictor import Predictor
predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/coref-model-2018.02.05.tar.gz")
predictor.predict(
  document="Paul Allen was born on January 21, 1953, in Seattle, Washington, to Kenneth Sam Allen and Edna Faye Allen. Allen attended Lakeside School, a private school in Seattle, where he befriended Bill Gates, two years younger, with whom he shared an enthusiasm for computers. Paul and Bill used a teletype terminal at their high school, Lakeside, to develop their programming skills on several time-sharing computer systems."
)